# Dependencies and imports

In [1]:
# %pip install langchain
# %pip install transformers
# %pip install python-decouple
# %pip install yandexcloud
# %pip install spacy
# %pip install nltk
# %pip install tiktoken

In [72]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import SpacyTextSplitter
from langchain.text_splitter import NLTKTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.schema.document import Document
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.chains import LLMChain
from langchain_core.load.serializable import Serializable
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.utils import enforce_stop_tokens

from decouple import config
from tqdm import tqdm
from typing import Any, List, Mapping, Optional
import json
import requests
import pandas as pd
import itertools

# Helper classes and functions

In [3]:
class _BaseCustomYandexGPT(Serializable):
    iam_token: str = ""
    api_key: str = ""
    x_folder_id: str = ""
    finetuned_model_id: str = ""
    model_name: str = "general"
    temperature: Optional[float] = 0.7
    max_tokens: int = 7400
    stop: Optional[List[str]] = None
    url: str = "https://llm.api.cloud.yandex.net/llm/v1alpha/instruct"
    debug_prints = False

    @property
    def _llm_type(self) -> str:
        return "yandex_gpt"


class YandexCustomGPT(_BaseCustomYandexGPT, LLM):
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {
            "model_name": self.model_name,
            "temperature": self.temperature,
            "finetuned_model_id": self.finetuned_model_id,
            "max_tokens": self.max_tokens,
            "stop": self.stop,
        }

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        iam_token_for_auth = self.finetuned_model_id or not self.api_key
        if iam_token_for_auth:
            auth_string = f"Bearer {self.iam_token}"
        else:
            auth_string = f"Api-Key {self.api_key}"
        headers = {"Authorization": auth_string}
        if self.x_folder_id and iam_token_for_auth:
            headers["x-folder-id"] = f"{self.x_folder_id}"
            
        json_body = {"model": "general",
                     "request_text": prompt,
                     "generation_options": {"max_tokens": self.max_tokens, "temperature": self.temperature}
                    }
        
        if self.finetuned_model_id:
            json_body["instruction_uri"] = f"ds://{self.finetuned_model_id}"
            
        if self.debug_prints:
            print("Auth: {}, temperature: {}, prompt size: {}, finetuned model used: {}".format("IAM" if iam_token_for_auth else "API", 
                                                                                                self.temperature, 
                                                                                                len(prompt),
                                                                                                self.finetuned_model_id != ""))
        
        while True:
            try:
                result = requests.post(
                    url=self.url,
                    headers=headers,
                    json=json_body
                )
                text = json.loads(result.text)['result']['alternatives'][0]['text']
                if stop is not None:
                    text = enforce_stop_tokens(text, stop)
                break
            except:
                if self.debug_prints:
                    print(result.content)
                # time.sleep(1)
        return text

In [4]:
def get_summary(text, text_splitter, chain):    
    docs = [Document(page_content=text, metadata={"source": "local"})]
    split_docs = text_splitter.split_documents(docs)
    result = chain({"input_documents": split_docs}, return_only_outputs=True)
    return result["output_text"]

def get_summary_score(article_text, summary_text, api_key):
    full_input = f"Текст статьи:\n{article_text}\n\nКраткое содержание:\n{summary_text}"

    resp = requests.post(
        url="https://node-api.datasphere.yandexcloud.net/classify", 
        json={
            "Text": full_input,
        },
        headers={
            "Authorization": f"Api-Key {api_key}",
            "x-node-alias": "datasphere.user.yagpt-seminar-hw",
        }
    )
    return json.loads(resp.text)["Scores"][0]

In [13]:
def get_text_splitter(text_splitter_name, separator, chunk_size):
    if text_splitter_name == "spacy":
        return SpacyTextSplitter(
            chunk_size=chunk_size, 
            chunk_overlap=0,  
            separator=separator, 
            pipeline="sentencizer"
        )
    elif text_splitter_name == "character":
        return CharacterTextSplitter(
            separator=separator,
            chunk_size=chunk_size // 10,
            chunk_overlap=0,
        )
    elif text_splitter_name == "nltk":
        return NLTKTextSplitter(
            chunk_size=chunk_size, 
            chunk_overlap=0,  
            separator=separator
        )

In [6]:
def get_summarization_chain(chain_name, llm):
    prompt_template = (
        "На основании текста ниже, представляющего собой статью на habr, составь краткое содержание этого текста: "
        "{text}")
    prompt = PromptTemplate.from_template(prompt_template)

    if chain_name == "refine":
        refine_template = (
            "Твоя задача - финализировать краткое содержание.\n"
            "Мы уже составили существующее краткое содержание до определенного момента: {existing_answer}\n"
            "У нас есть возможность улучшить существующее краткое содержание "
            "(только если это необходимо) с некоторым дополнительным контекстом ниже.\n"
            "------------\n"
            "{text}\n"
            "------------\n"
            "Учитывая этот новый контекст, улучши оригинальное краткое содержание на русском языке."
            "Если контекст не приносит пользы, вернись к оригинальному краткому содержанию."
        )
        refine_prompt = PromptTemplate.from_template(refine_template)

        return load_summarize_chain(
            llm=llm,
            chain_type="refine",
            question_prompt=prompt,
            refine_prompt=refine_prompt,
            return_intermediate_steps=True,
            input_key="input_documents",
            output_key="output_text",
        )
    elif chain_name == "refine_small_prompt":
        refine_small_template = (
            "Твоя задача - улучшить краткое содержание, которое мы уже составили до определенного момента: \n"
            "{existing_answer}\n"
            "У нас есть возможность улучшить это краткое содержание с "
            "некоторым дополнительным контекстом представленным ниже. "
            "Учитывая этот дополнительный контекст, попробуй улучшить наше краткое содержание. "
            "Если представленный контекст не приносит пользы, то не используй его:\n"
            "{text}\n"
        )
        refine_small_prompt = PromptTemplate.from_template(refine_small_template)

        return load_summarize_chain(
            llm=llm,
            chain_type="refine",
            question_prompt=prompt,
            refine_prompt=refine_small_prompt,
            return_intermediate_steps=True,
            input_key="input_documents",
            output_key="output_text",
        )
    elif chain_name == "map_reduce":
        combine_template = (
            "Ниже приведен набор кратких содержаний:\n"
            "{text}"
            "Используй их чтобы получить одно дистиллированное краткое содержание, охватывающее основные приведенные темы. ")
        combine_prompt = PromptTemplate.from_template(combine_template)

        return load_summarize_chain(
            llm=llm,
            chain_type="map_reduce",
            map_prompt=prompt,
            combine_prompt=combine_prompt,
            return_intermediate_steps=True,
            input_key="input_documents",
            output_key="output_text",
        )

# Parameters initialization

In [7]:
api_key = config("YC_API_KEY")
iam_token = config("YC_IAM_TOKEN")
x_folder_id = config("YC_X_FOLDER_ID")
summary_small_4k_model_id = config("SUMMARY_SMALL_4K_ID")
summary_small_4k_bad_prompt_id = config("SUMMARY_SMALL_4K_BAD_PROMPT_ID")
summary_mid_4k_id = config("SUMMARY_MID_4K_ID")

In [112]:
temperature_values = [0.3, 0.65]
finetuned_models = {
    "": "", 
    "small": summary_small_4k_model_id, 
    "small_bad_prompt": summary_small_4k_bad_prompt_id, 
    "mid": summary_mid_4k_id
}
chain_strings = ["map_reduce", "refine", "refine_small_prompt"]
text_splitters = ["character", "spacy", "nltk"]
chunk_separators = ["\n\n", ". "]
chunk_sizes = [3000, 100000]
retries_cnt = 1
debug_prints = False

In [58]:
with open('test_articles_clear_100.json', 'r') as fin:
    test_articles = [json.loads(s) for s in fin.readlines()]

# Langchain summarization chain approach

In [ ]:
for i in range(retries_cnt):
    print("Doing {}/{} pass".format(i + 1, retries_cnt))
    for temperature, model_name, chain_name, \
        text_splitter_name, separator, chunk_size in itertools.product(temperature_values, finetuned_models.keys(), 
                                                                       chain_strings, text_splitters, chunk_separators, chunk_sizes):
        llm = YandexCustomGPT(api_key=api_key, iam_token=iam_token, x_folder_id=x_folder_id, 
                              finetuned_model_id=finetuned_models[model_name], temperature=temperature, debug_prints=debug_prints)
        chain = get_summarization_chain(chain_name, llm)
        text_splitter = get_text_splitter(text_splitter_name, separator, chunk_size)
        scores = []
        for test_article in tqdm(test_articles, desc='Articles'):
            article_text = test_article["Text"]
            article_id = test_article["id"]
            summary_text = get_summary(article_text, text_splitter, chain)
            summary_score = get_summary_score(article_text, summary_text, api_key)
            result = {"id": article_id, 
                      "text": article_text, 
                      "text_size": len(article_text),
                      "summary": summary_text, 
                      "summary_size": len(summary_text),
                      "score": summary_score,
                      "temperature": temperature,
                      "finetune_model_name": model_name,
                      "summarization_chain": chain_name,
                      "text_splitter_name": text_splitter_name,
                      "chunk_separator": separator,
                      "chunk_size": chunk_size,
                     }
            scores.append(summary_score)
            if debug_prints:
                print({k: v for k, v in result.items() if k not in ["text", "summary"]})
            with open('multiparameter_results_jsonlines.json', 'a', encoding='utf8') as fout:
                fout.write(json.dumps(result, ensure_ascii=False) + "\n")
        print("Scores are: {}\n for the config: {}".format(scores, {k: v for k, v in result.items() if k not in ["id", "text", "summary"]}))

Doing 1/1 pass


Articles: 100%|██████████| 100/100 [15:34<00:00,  9.34s/it]

Scores are: [0.624512, 0.452637, 0.461182, 0.584473, 0.480225, 0.401367, 0.402344, 0.273438, 0.383301, 0.332764, 0.36084, 0.211548, 0.510254, 0.517578, 0.547363, 0.668945, 0.437256, 0.529297, 0.522461, 0.404785, 0.489258, 0.468018, 0.393066, 0.459229, 0.51709, 0.52832, 0.465332, 0.633789, 0.331543, 0.437988, 0.434082, 0.313232, 0.334229, 0.585449, 0.584961, 0.286377, 0.487061, 0.577148, 0.249634, 0.481445, 0.475098, 0.346924, 0.415527, 0.609375, 0.277344, 0.497559, 0.372803, 0.256836, 0.420898, 0.494629, 0.614746, 0.426758, 0.570313, 0.655762, 0.328613, 0.415771, 0.54248, 0.597168, 0.305664, 0.652344, 0.390869, 0.393799, 0.455078, 0.284424, 0.519531, 0.641113, 0.506836, 0.668457, 0.634277, 0.616699, 0.588867, 0.561523, 0.546387, 0.228516, 0.646973, 0.562988, 0.307617, 0.521484, 0.378418, 0.543945, 0.426025, 0.480225, 0.59375, 0.15332, 0.26123, 0.384277, 0.432373, 0.335449, 0.451904, 0.509277, 0.682129, 0.188354, 0.52002, 0.446045, 0.634277, 0.583008, 0.498535, 0.619629, 0.694336, 0.700

# Few-shot/zero-shot approach

In [113]:
for i in range(retries_cnt):
    print("Doing {}/{} pass".format(i + 1, retries_cnt))
    temperature = 0.25
    model_name = ""
    llm = YandexCustomGPT(api_key=api_key, iam_token=iam_token, x_folder_id=x_folder_id, 
                          finetuned_model_id=finetuned_models[model_name], temperature=temperature, debug_prints=debug_prints)
    
    first_example = (
        "Текст:\n"
        "Физика в мире животных: птицы-ныряльщики и их система безопасности / Habr            \n\n"
        "08-10-2016\n"
        "Некоторые виды морских птиц способны нырять на большие глубины с очень высокой начальной скоростью. Некоторые виды птиц пикируют с высоты, погружаясь в воду без вреда для себя. Человек, если бы погружался в воду на такой же скорости, получил бы очень серьезные повреждения внутренних органов и, с высокой вероятностью, умер бы. Но птицы без проблем переносят быстрое погружение в глубину. Этим отличаются, например, олуши. Олуши — крупные птицы, свободно парят над океаном и добывают рыбу и кальмаров, ныряя в воду с высоты 10—100 м на глубину до 25 м. Длина тела взрослых особей рода Sula составляет 71—91 см, размах крыльев — до 1,5 м, вес 0,7—1,5 кг. При нырянии могут развивать скорость у поверхности воды до 140 км/ч. Как образом эти птицы способны на подобное? У олушей есть несколько защитных механизмов слухового и зрительного аппаратов и всего тела. Завершенное недавно исследование группы ученых Виргинского политехнического института прояснило ситуацию. Резульатты были опубликованы в Proceedings of the National Academy of Sciences. «Нам было интересно, что происходит с птицами, когда они погружаются в воду. Олуши показали просто невероятный результат», — говорит Санни Джанг (Sunny Jung), один из участников команды исследователей. Исследователи в ходе своей работы изучили биомеханику олушей во время ныряния. Они обнаружили, что форма головы птиц, длина шеи и особенности мускулатуры шеи эффективно сопротивляются начальному сопротивлению и растущему с глубиной давлению столба воды, предотвращая негативное воздействие высокого давления на тонкую шею и все тело птицы. Основная сила, действующая на голову птицы в начале погружения — это сопротивление. Его олуши без проблем научились преодолевать. Предыдущие исследования фокусировались на экологических аспектах хищнических инстинктов олушей. Джанг с коллегами решили изучить механизм погружения олушей в воду. Группа ученых первой выяснила многие детали физики погружения, включая и особенности строения тела, которые позволяют олушам отлично чувствовать себя на большой глубине. Ученые создали модель тела и шеи (с мускулатурой). Также специалисты распечатали на 3D принтере копии черепов олушей. Копии был созданы на основе настоящих черепов, предоставленных Смитсониановским институтом. Благодаря «распечатке» ученые смогли измерить силы, которые действуют на голову и шею птицы во время нырка. Для дополнительного изучения возможностей олучшей ученые создали 3D-модель головы и шеи олуши. Модель, было решено сделать упрощенной, постепенно усложняя в ходе работы. Голову надели на гибкую прорезиненную «шею». Всю систему затем несколько раз погружали в воду, испытывая процесс ныряния олушей. «Голову» и «шею» погружали на разную глубину под разными углами. Искусственную шею покрывали разными материалами с различными свойствами. Процесс погружения снимали на высокоскоростную камеру, которая и помогла установить истину. «Это именно то, что мы сделали. Нашли сложную систему и решили ее упростить», — говорит Браян Чанг (Brian Chang) из Ваншингтонского университета. Результат анализа показал, что, в зависимости от геометрии головы, сильное давление на нее может не оказать почти никакого влияния, а может и сильно повредить здоровью птицы. Также ученые изучали прочность шеи, которая способна вынести огромные нагрузки. «Мы обнаружили, что у всех олуш голова видоизменена особым образом, что характерно для других птиц семейства. Это снижает влияние толщи воды на череп. Как оказалось, причина еще и в том, что олуши перед погружением сильно сокращают мышцы шеи, что не приводит к смерти этих птиц. Сокращая мышцы шеи, олуши снижают влияние сопротивления воды на организм. Биомеханика позволила ученым создать модели скелетов не только олушей, о которых идет речь в этой статье. „В своем исследовании мы сконцентрировались на олушах и бакланах, которые демонстрируют настоящую фантастику, когда дело доходит до охоты при помощи ныряния“. Выяснилось, что большое значение для преодоления сопротивления воды имеет форма головы, свойства материала шеи, скорость входа в воду. При нырке тело олуши принимает вытянутую форму, птица входит в воду, как стрела, выпущенная из лука. Тело олушей идеально приспособлено для скоростного ныряния. Интересно, что у других представителей семейства способности нырять в воду, срываясь с высоты и пикируя нет. У человека, к сожалению, нет возможности защититься при нырянии на глубину за считанные минуты, как это делают птицы. Но нырять с высоты многим людям нравится. Сейчас все более популярными становятся такие виды экстремального спорта, как прыжки с воду с большой высоты — с мостов, утесов и других высоких объектов. Результаты исследования, полученные учеными, помогут разработать рекомендации для экстремалов, позволяющие избежать повреждений при входе в толщу воды. «Мы считаем, что наши выводы могут использовать реальные хай-дайверы», — говорят ученые.\n\n"
        "Краткое содержание:\n"
        "[SEP]\n"
        "Статья рассказывает о том, как птицы-ныряльщики, такие как олуши, способны погружаться в воду на большие глубины с высокой скоростью без вреда для себя. Ученые провели исследование, чтобы понять, как это происходит. Они обнаружили, что форма головы и шеи птиц, а также особенности их мускулатуры эффективно сопротивляются начальному сопротивлению и давлению воды, предотвращая негативное воздействие на тело птицы. Это исследование поможет разработать рекомендации для экстремалов, чтобы избежать повреждений при входе в воду.                                               \n\n08-02-2018\nДоход от вложений в разные активы по итогам 2017 года Компания CoinDesk опубликовала большой аналитический отчёт State of Blockchain 2018, в котором анализирует состояние рынка криптовалют в IV кв. 2017 года и приводит интересную рыночную статистику по публичным блокчейнам, консорциумным блокчейнам, распределённым реестрам (DLT), первичным размещениям монет (ICO), трейдингу и инвестициям. Также перечисляются попытки правительств разных стран регулировать этот сектор новой экономики. Сами автора отчёта выделяют пять главных тем последнего времени. 1. Пузырь или нет Рост курса биткоина в 2017 году на 1278% (то есть в 13,8 раз) заставил экспертов в очередной раз спорить, можно ли называть биткоин «пузырём», то есть активом с неоправданно раздутой стоимостью. Такая дилемма разделила и читателей CoinDesk: в опросе 49% ответили «да», а 39% — «нет». Аналитические данные из отчёта позволяют более обоснованно обсуждать эту проблему. Например, такой факт: согласно опросу, среди всех криптоинвесторов только 19% брали кредиты, чтобы купить криптовалюту в период её бурного роста. А из тех, кто брал кредиты, более половины уже вернули долг. Это важный показатель, потому что в исторической перспективе известно, что финансовые пузыри в значительной степени накачиваются именно за заёмные деньги на волне хайпа. Так что если и считать биткоин пузырём, то это явно не такой пузырь, какие мы встречали в прошлом, считают авторы отчёта из CoinDesk (тут нужно заметить, что компания CoinDesk глубоко погружена в бизнес с криптовалютами, так что никак не является нейтральной стороной в проведении этого опроса и интерпретации его результатов). Маржинальная торговля с кредитным плечом на криптобиржах появилась относительно недавно и не получила особого распространения. 2. Диверсификация рынка Другая важная тенденция — диверсификация рынка. В январе 2017 года капитализация биткоина превышала 90% от общей капитализации всех криптовалют, а доля Ethereum была ничтожной. Но всё изменилось с выходом «киллер-фичи» — смарт-контрактов ERC-20, которые позволяли генерировать токены и проводить ICO. Резко вырос спрос на эфир (обычно необходимый для участия в ICO) — и в третьем квартале 2017 года с доминированием биткоина было покончено. Конец доминирования биткоина совпал по времени с активацией протокола Segregated Witness (SegWit), которая в сети Bitcoin произошла 24 августа 2017 года. 3. Рекорды Ethereum Бум на ICO стимулировал спрос на эфир в 2017 году, но не только он. Были и другие феноменально популярные проекты. Например, декабрьская истерия вокруг «криптокотят» CryptoKitties — своеобразных децентрализованных тамагочи. Хотя многие критиковали глупость самой игры, она стимулировала дополнительный интерес широкой публике к блокчейну и эфиру. Хотя во втором и третьем кварталах Ethereum ставил абсолютные рекорды по количеству транзакций, в IV кв. 2017 года интерес к новой игре и обновления после жёсткого форка на Byzantium подтолкнули сеть к новым рекордным достижениям — и количество транзакций опять удвоилось квартал к кварталу. По итогам октября-декабря среднее количество транзакций в сети Ethereum составило 589 171. Для сравнения, в сети Bitcoin — 319 085. Таким образом, Ethereum сохраняет лидерство среди всех криптовалют по количеству транзакций. К сожалению, вместе с ростом количества транзакций и ростом биржевого курса увеличивалась и средняя комиссия за транзакции в сетях Bitcoin и Etherium. В сети Bitcoin в конце прошлого года размер средней комиссии вообще подскочил до умопомрачительного значения $13,20 за транзакцию, хотя ещё в начале года было $0,62. 4. Корея заняла место Китая Прошлый год в Китае прошёл под знаком государственных запретов. В сентябре власти запретили проведение ICO, а потом и вовсе закрыли все криптобиржи. Однако рынок быстро заполнил пустоту и не особенно пострадал от закрытия китайских бирж. Торги просто переместились на другие площадки, в том числе в Южную Корею. Курс тоже не обвалился — наоборот, после закрытия китайских бирж случилось рекордное ралли с почти вертикальным ростом биткоина. 4. Раздача денег Хотя в 2017 году прошло много крупных ICO, но основную роль в повышении общей капитализации рынка криптовалют сыграли не они, а форки с бесплатной раздачей денег «ходлерам» (постоянным держателям монет). Первой раздачу монет провела сеть Bitcoin Cash: каждый ходлер биткоинов получил аналогичное количество монет BCH. Примеру BCH вскоре последовала система Stellar. И немного оптимизма напоследок. Среди инвесторов, опрошенных CoinDesk, лишь 6,2% сказали, что в 2017 году стали криптомиллионерами. Зато 38,8% думают, что у них получится это в 2018 году.\n\n\nКраткое содержание:\nКомпания CoinDesk опубликовала аналитический отчёт State of Blockchain 2018, в котором рассматривает состояние рынка криптовалют в конце 2017 года. В опросе было выяснено, что среди всех криптоинвесторов только 19% брали кредиты, чтобы купить криптовалюту в период её бурного роста, и более половины из них уже вернули долг. Ethereum ставил рекорды по количеству транзакций, а криптовалютный рынок стал диверсифицироваться. Запреты в Китае на ICO и криптобиржи привели к тому, что торги переместились в Южную Корею. Также форки с бесплатной раздачей денег играли важную роль в повышении капитализации рынка криптовалют.'\n"
        "[SEP]\n\n"
    )
    
    second_example = (
        "Текст:\n"
        "Дайджест последних достижений в области криптографии. Выпуск первый / Хабр               \n\n"
        "13-07-2016\n"
        "Привет, %username% Пришло время для свежей пачки криптоновостей, пока они еще не перестали быть новостями. В этом выпуске: Новый рекорд вычисления дискретного логарифма VPN сервер и клиент, использующие Noise протокол Постквантовая криптография в Chrome уже сегодня! Чего вы не знаете о новом E2E шифровании в Facebook RLWE избавляется от R и это идет ему на пользу Comodo хотел поиметь Let`s Encrypt, но сфейлил. А Let`s Encrypt с завтрашнего дня будет поддерживать ddns Появились минимальные требования к реализациям алгоритмов RSA, DSA, DH, устойчивым к side-channel атакам Предыдущий выпуск тут Рекорд вычисления дискретного логарифма Группа исследователей из EPFL и Университета Лейпцига смогла посчитать логарифм по основанию простого числа размером 768 бит. Для этого им понадобилось 200 ядер и время с февраля 2015 года. Использовали они вариант цифрового решета. Таким образом логарифмирование сравнялось с факторизацией где рекорд для обычных чисел тоже 768 бит Wireguard. VPN, использующий самые модные криптоалгоритмы Не успели опубликовать спеку по Noise protocol, как уже появилось решение на его основе. Очень минималистичный VPN, в котором используются Noise protocol framework, Curve25519, ChaCha20, Poly1305, BLAKE2, SipHash24 и HKDF. Работает в режиме ядра, но активно пилятся usermode версии на Go и Rust. Советую присмотреться, очень крутая штука. Google добавил постквантовое шифрование в Chrome Canary Подробнее тут. Используется алгоритм New Hope, который основан на проблеме RLWE, которая в свою очередь является частным вариантом криптографии на решетках. Это сравнительно молодая область криптографии, еще плохо изучена и поэтому её пока нельзя использовать в реальной жизни. Но в качестве эксперимента, почему бы и нет? E2E шифрование в Facebook позволяет настучать на собеседника Они назвали этот механизм Franking. Он позволяет отправить Abuse report в случае необходимости. Реализуется следующим образом: Генерируется случайный ключ Nf Считается Tf = HMAC·SHA256(Nf, M) Nf конкатенируется с M и шифруется ключом получателя. На сервер отправляется Tf и шифротекст Сервер считает Rf= HMAC·SHA256(ключ фейсбука, Tf || метаданные (кто, кому,...)) Получателю отправляется Rf, Tf, шифротекст Получатель расшифровывает шифротекст, считает HMAC(Nf, M) и сравнивает с Tf. Если сравнение не проходит, то выбрасывает сообщение Если получатель хочет пожаловаться фейсбуку, то отправляет ему расшифрованное сообщение, Rf, Nf Фейсбук убеждается, что это именно то сообщение, которое отправил отправитель и принимает соответствующие действия Таким образом, отвертерться от обзывательства Васи козлом не получится. RLWE без R Итак, есть криптография на решетках. Хороша тем, что её в будущем не взломает квантовый компьютер. Но её параметры огромны, размеры ключей достигают мегабайтов. Есть частный её случай, называемый обучение с ошибками. Так вот, обучение с ошибками хоть тоже очень круто, но тем не менее из-за ограничений на размер ключа и других было нереально использовать в продакшене. Поэтому к LWE добавили кольца и назвали это RLWE, который уже используется в Chrome Canary, т.е. там параметры стали уже более-менее человеческими по размерам. К сожалению, степень изученности обратно пропорциональна навороченности алгоритма и добавление колец возможно ослабило LWE. Поэтому группа товарищей реализовала согласование ключей без колец и опубликовало на эту тему доку. Размеры сообщений в каждую сторону находятся в пределах 12 кб, операция согласования ключей занимает около 1.3 мс. Это примерно в 5 раз больше по объему хэндшейка DH, а так же в 1.6 раз замедляет пропускную способность TLS сервера, но тем не менее это уже сравнимо с New Hope и можно использовать на практике. При этом структура получается более безопасной. Comodo оборзел И решил для нескольких своих сервисов зарегистрировать торговую марку Let’s Encrypt. Мало того, что торгуют воздухом, так еще и чужая слава покоя не дает. Но, сообщество собралось силами, надавало комоду по сусалам и отбило торговую марку. Подробности тут. Кстати, после завтрашнего апдейта можно будет прикручивать бесплатный TLS к dyndns хостам! Это суперкруто, все хомяки теперь будут с сертификатами. Защищаемся от Side channel атак Не секрет, что нынче информацию о ключах шифрования можно удаленно снимать чуть ли не через вентилятор. Поэтому, все большую популярность обретают constant-time алгоритмы, которые не зависят от входных данных. Немцы выпустили минимальные требования для реализаций, выполнение которых усложнит задачу получения секретных данных через побочные каналы данных. Интересный документ, советую ознакомиться. На этом у меня всё, до новых встреч!\n\n"
        "Краткое содержание:\n"
        "[SEP]\n"
        "В этом выпуске криптоновостей: \n 1. Рекорд вычисления дискретного логарифма: Группа исследователей из EPFL и Университета Лейпцига смогла посчитать логарифм по основанию простого числа размером 768 бит за 200 ядер.\n 2. VPN на основе Noise protocol: Опубликовано решение на его основе, использующее популярные криптоалгоритмы.\n 3. Постквантовое шифрование в Chrome Canary: Google добавил алгоритм New Hope, основанный на проблеме RLWE.\n 4. E2E шифрование в Facebook: Реализован механизм Franking для отправки Abuse report.\n 5. RLWE без R: Группа товарищей реализовала согласование ключей без колец.\n 6. Comodo vs Let's Encrypt: Сообщество отбило торговую марку Let's Encrypt у Comodo.\n 7. Защищаемся от Side channel атак: Выпущены минимальные требования для реализаций алгоритмов, устойчивых к атакам через побочные каналы.\n"
        "[SEP]\n\n"
    )
    
    template = first_example + second_example + (
        "Текст:\n"
        "{text}\n\n"
        "Краткое содержание:\n"
        "[SEP]\n"
    )
    prompt = PromptTemplate(template=template, input_variables=["text"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    scores = []
    for test_article in tqdm(test_articles, desc='Articles'):
        article_text = test_article["Text"]
        article_id = test_article["id"]
        
        summary_text = llm_chain.run(article_text).replace("[SEP]", "")
        summary_score = get_summary_score(article_text, summary_text, api_key)
        result = {"id": article_id, 
                  "text": article_text, 
                  "text_size": len(article_text),
                  "summary": summary_text, 
                  "summary_size": len(summary_text),
                  "score": summary_score,
                  "temperature": temperature,
                  "finetune_model_name": "",
                  "summarization_chain": "two-shot",
                  "text_splitter_name": "",
                  "chunk_separator": "",
                  "chunk_size": "",
                 }
        scores.append(summary_score)
        with open('multiparameter_results_jsonlines.json', 'a', encoding='utf8') as fout:
            fout.write(json.dumps(result, ensure_ascii=False) + "\n")
    print("Scores are: {}\n for the config: {}".format(scores, {k: v for k, v in result.items() if k not in ["id", "text"]}))

Doing 1/1 pass


Articles: 100%|██████████| 100/100 [04:10<00:00,  2.51s/it]

Scores are: [0.316895, 0.304688, 0.570801, 0.543457, 0.714355, 0.583984, 0.632324, 0.544922, 0.556152, 0.620117, 0.373779, 0.264648, 0.710449, 0.459229, 0.635254, 0.587402, 0.359131, 0.51416, 0.386475, 0.602051, 0.415527, 0.609863, 0.293945, 0.631348, 0.434082, 0.408691, 0.529785, 0.531738, 0.497314, 0.620605, 0.568359, 0.618164, 0.417969, 0.55127, 0.672852, 0.674316, 0.551758, 0.5, 0.313232, 0.533691, 0.557617, 0.418945, 0.598145, 0.549316, 0.620117, 0.555176, 0.556641, 0.506836, 0.525879, 0.645996, 0.448975, 0.561035, 0.619141, 0.514648, 0.396973, 0.523438, 0.478027, 0.45459, 0.547363, 0.57666, 0.462402, 0.547363, 0.555176, 0.51123, 0.39209, 0.583496, 0.444824, 0.63623, 0.512207, 0.586426, 0.505859, 0.611328, 0.567871, 0.397949, 0.648926, 0.491699, 0.280029, 0.581543, 0.463867, 0.491211, 0.630859, 0.381836, 0.516602, 0.506348, 0.431885, 0.500488, 0.503418, 0.498291, 0.378662, 0.556152, 0.633301, 0.32666, 0.608887, 0.492432, 0.555176, 0.404053, 0.533203, 0.681152, 0.651367, 0.691895]


# Results analysis

In [116]:
df = pd.read_json("multiparameter_results_jsonlines.json", lines=True)

In [121]:
filtered_df = df[((df["finetune_model_name"] == "small_bad_prompt") & 
                  ((df["summary_size"] >= 450) | (df["score"] <= 0.5))) | 
                 (df["finetune_model_name"] != "small_bad_prompt")]

top_n = 3
max_scores_top3_df = filtered_df.groupby('id').apply(lambda x: x.nlargest(top_n, 'score')).reset_index(drop=True)

In [131]:
def select_row(group_df):
    first_row = group_df.iloc[0]
    second_row = group_df.iloc[1]
    third_row = group_df.iloc[2]
    
    llm = YandexCustomGPT(iam_token=iam_token, x_folder_id=x_folder_id, temperature=0.2, debug_prints=debug_prints)
    
    template = (
        "Ниже представлены 3 версии краткого содержания одного и того же текста, необходимо выбрать номер (одна цифра 1, 2 или 3) лучшего из них:\n"
        "Краткое содержание #1:\n"
        "В подборке:\n - Советы по Python: итерирование нескольких итерируемых объектов с помощью zip.\n - Генераторы и закрытие: три аспекта работы с генераторами.\n - f-строки: форматирование строк с вычисленными выражениями.\n - Метаклассы и производные классы.\n - Использование __init__ для модификации объекта сразу после создания.\n\n"
        "Краткое содержание #2:\n"
        "В подборке советов про Python и программирование:\n - Как итерировать несколько итерируемых объектов с помощью функции zip. \n - Как остановить генератор с помощью метода close. \n - Три аспекта, связанных с остановкой генератора при обработке GeneratorExit. \n - Как использовать f-строки для форматирования значений. \n - Как создать класс с метаклассом, производным от другого класса. \n - Как автоматически получить метакласс другого класса с помощью функции create_my_dict_class. \n - Как использовать метод __init__ для модификации объекта сразу после его создания. \n - Как использовать cached_property для кеширования вычисленных значений.\n\n"
        "Краткое содержание #3:\n"
        "Новая подборка советов про Python и программирование из авторского канала @pythonetc. В статье рассматриваются следующие темы: итерация нескольких итерируемых объектов с помощью функции zip, использование генератора для остановки и возобновления генерации значений, применение f-строк для форматирования вывода, создание производных классов с использованием метаклассов, использование метода __new__ для контроля создания объектов, а также использование метода @cached_property для кэширования значений.\n\n"
        "Номер лучшего краткого содержания:\n"
        "[SEP]\n"
        "2\n"
        "[SEP]\n"
        "Ниже представлены 3 версии краткого содержания одного и того же текста, необходимо выбрать номер (одна цифра 1, 2 или 3) лучшего из них:\n"
        "Краткое содержание #1:\n"
        "Статья рассказывает о Центре молодежного инновационного творчества \"LAB3DPrint\", который предоставляет услуги по прототипированию, 3D-печати и обучению работе на цифровом оборудовании. Интервью с Иваном Мошкиным, руководителем Лаборатории трехмерной печати, обсуждает выбор модели 3D-принтера Raise3D N2 Plus Dual, его впечатления о принтере и основные направления деятельности центра.\n\n"
        "Краткое содержание #2:\n"
        "В гостях у Центра молодежного инновационного творчества «LAB3DPrint» побывали представители компании «Цветной мир», чтобы обсудить впечатления от использования 3D-принтера Raise3D N2 Plus. Директор центра Иван Мошкин рассказал о выборе принтера, его плюсах и минусах, а также о направлениях работы центра. Компания «Цветной мир» предлагает 10 преимуществ при покупке 3D-оборудования, включая гарантию, бесплатную доставку, бесплатное обучение и возможность купить принтер в кредит.\n\n"
        "Краткое содержание #3:\n"
        "В интервью с главой ЦМИТа «LAB3DPrint» Иваном Мошкиным обсуждались вопросы о том, чем живет компания, какие успехи удалось достичь и его впечатления о Raise3D.\n\n Рассказано о том, почему был выбран принтер Raise3D N2 Plus Dual, о его преимуществах и недостатках.\n\n Описываются направления деятельности ЦМИТа и успехи учеников в различных конкурсах.\n\n Дается информация о возможности приобретения принтера и других преимуществах сотрудничества с компанией «Цветной Мир».\n\n"
        "Номер лучшего краткого содержания:\n"
        "[SEP]\n"
        "1\n"
        "[SEP]\n"
        "Ниже представлены 3 версии краткого содержания одного и того же текста, необходимо выбрать номер (одна цифра 1, 2 или 3) лучшего из них:\n"
        "Краткое содержание #1:\n"
        "Руководство FTX потратило более \$40 млн на роскошные курорты, еду и перелёты за последние 9 месяцев работы криптобиржи. \n\n Сэм Бэнкман-Фрид и его коллеги из FTX Digital Markets не жалели средств инвесторов на роскошные отели, дорогие авиаперелёты и рестораны. \n\n Прокуратура США выяснила, что родители основателя FTX и высшее руководство компании приобрели не менее 19 объектов недвижимости на Багамах. \n\n Сэм Бэнкман-Фрид арестован по запросу США в своём доме на Багамах. Багамский суд отказал ему в освобождении под залог. \n\n Суд на Багамах одобрил экстрадицию Бэнкмана-Фрида в США. \n\n Комиссия по ценным бумагам и биржам США обвинила основателя FTX в обмане инвесторов. \n\n Сэм Бэнкман-Фрид признал себя виновным по всем обвинениям и сотрудничает с прокуратурой. \n\n Сэма Бэнкмана-Фрида после экстрадиции в США освободили из под стражи под залог в \$250 млн. \n\n Клиенты обанкротившейся FTX подали коллективный иск в суд. \n\n Эксперт обнаружил, что находясь под домашним арестом, Сэм Бэнкман-Фрид обналичил \$684 тыс. в криптовалюте. \n\n Бэнкман-Фрид отказался признавать себя виновным по обвинениям в обмане инвесторов и причинении миллиардных убытков. \n\n Судебный процесс по делу против Бэнкмана-Фрида назначен на октябрь 2023 года. Предположительно, что суд вынесет приговор в ноябре. \n\n До этого времени Бэнкман-Фрид будет находится под домашним арестом.\n\n"
        "Краткое содержание #2:\n"
        "Руководство FTX потратило более \$40 млн на роскошные отели, еду и перелеты за последние 9 месяцев работы криптобиржи, выяснили СМИ на основании судебных документов. \n\n Сэм Бэнкман-Фрид и его коллеги из FTX Digital Markets не жалели средств инвесторов на роскошные отели, дорогие авиаперелеты и рестораны. \n\n Прокуратура США выяснила, что в течение последних двух лет родители основателя Бэнкмана-Фрида и высшее руководство FTX приобрели не менее 19 объектов недвижимости на Багамах. \n\n Бэнкман-Фрид признал недостатки надзора в FTX, но заявил, что лично он не считает, что несёт уголовную ответственность за совершённые деяния. \n\n Суд на Багамах одобрил экстрадицию в США основателя криптобиржи FTX Сэма Бэнкмана-Фрида. \n\n Комиссия по ценным бумагам и биржам США обвинила основателя криптобиржи FTX Сэма Бэнкмана-Фрида в обмане инвесторов. \n\n Сэм Бэнкман-Фрид отказался признавать себя виновным по уголовным обвинениям в обмане инвесторов криптобиржи и причинении им миллиардных убытков. \n\n Судебный процесс по делу против Бэнкмана-Фрида назначен на октябрь 2023 года. Предположительно, что суд вынесет приговор в ноябре. До этого времени Бэнкман-Фрид будет находится под домашним арестом.\n\n"
        "Краткое содержание #3:\n"
        "Руководство криптовалютной биржи FTX потратило более $40 млн на проживание и питание своих сотрудников, аренду апартаментов и перелеты. Основатель биржи Сэм Бэнкман-Фрид был экстрадирован в США, где суд отказался освободить его под залог. Бэнкман-Фрид отрицает обвинения в мошенничестве и надеется на оправдательный приговор. Прокуратура намерена добиваться отстранения Бэнкмана-Фрида от руководства компанией. Автор сомневается в том, что Бэнкману-Фриду удастся избежать наказания.\n\n"
        "Номер лучшего краткого содержания:\n"
        "[SEP]\n"
        "3\n"
        "[SEP]\n"
        "Ниже представлены 3 версии краткого содержания одного и того же текста, необходимо выбрать номер (одна цифра 1, 2 или 3) лучшего из них:\n"
        "Краткое содержание #1:\n"
        "{text1}\n\n"
        "Краткое содержание #2:\n"
        "{text2}\n\n"
        "Краткое содержание #3:\n"
        "{text3}\n\n"
        "Номер лучшего краткого содержания:\n"
        "[SEP]\n"
    )
    
    prompt = PromptTemplate(template=template, input_variables=["text1", "text2", "text3"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    best_summary_index = llm_chain.run(text1=first_row["summary"], text2=second_row["summary"], text3=third_row["summary"]).replace("[SEP]", "")
    max_score = max(first_row["score"], second_row["score"], third_row["score"])
    if best_summary_index.isnumeric() and int(best_summary_index) in [1, 2, 3]:
        if max_score - group_df.iloc[int(best_summary_index) - 1]["score"] <= 0.02 and group_df.iloc[int(best_summary_index) - 1]["score"] >= 0.53:
            return group_df.iloc[int(best_summary_index) - 1]
    # print("Model failed to choose the number")
    if first_row["score"] == max_score:
        return first_row
    if second_row["score"] == max_score:
        return second_row
    return third_row

tqdm.pandas()
max_score_df = max_scores_top3_df.copy().groupby('id').progress_apply(select_row).reset_index(drop=True)
#max_score_df = filtered_df.loc[filtered_df.groupby('id')['score'].idxmax()]

100%|██████████| 100/100 [02:17<00:00,  1.38s/it]


In [132]:
print("Mean score: {}".format(max_score_df["score"].mean()))
max_score_df.sort_values(by='score', ascending=True).head(20)

Mean score: 0.64289068


,id,text,text_size,summary,summary_size,score,temperature,finetune_model_name,summarization_chain,text_splitter_name,chunk_separator,chunk_size
55,62,Использование камеры Fish eye на Raspberry Pi ...,5812,"В данной статье рассказывается о том, как испо...",277,0.536133,0.30,small,refine,character,\n\n,100000
10,11,Разработка монолитной Unix подобной OS — Систе...,5329,В этой статье рассказывается о разработке сист...,440,0.536133,0.20,mid,refine,character,\n\n,100000
81,89,Установка Fedora 16 в качестве полнофункционал...,3254,"В данной статье рассказывается о том, как уста...",445,0.536133,0.25,,two-shot,,,
88,96,Как превратить медиаплеер в неттоп? / Хабр ...,4626,"Статья рассказывает о том, как превратить меди...",446,0.541992,0.70,mid,refine,character,\n\n,100000
22,24,Редактор сценариев Age of Empires 2 можно прев...,5966,"Автор статьи рассказывает о том, как использов...",479,0.542480,0.30,small_bad_prompt,refine,character,\n\n,100000
38,43,Dagaz: Конец одиночества / Хабр ...,6572,"В статье рассказывается о проекте Dagaz, котор...",327,0.543945,0.35,small,refine_small_prompt,spacy,\n\n,100000
64,71,Пишем Grafana reverse proxy на Go / Habr ...,7367,Обратный прокси на Go в три строчки — это стат...,368,0.546875,0.30,small,refine,character,\n\n,100000
63,70,habr.com\n== Перевод из любой системы счислени...,4403,Автор статьи рассказывает о создании консольно...,314,0.551270,0.70,mid,refine,character,\n\n,100000
84,92,"Имитируем адаптацию глаза к темноте в 3D, или ...",6431,В данном тексте описывается процесс имитации а...,757,0.551270,0.30,small_bad_prompt,refine,character,\n\n,100000
36,41,Настройка Reverse Proxy Apache (Debian 8) с ав...,5440,В этой статье рассказывается о настройке Rever...,325,0.551758,0.25,,two-shot,,,


In [133]:
results = max_score_df[["id", "summary"]].sort_values(by='id', ascending=True).reset_index(drop=True).to_dict('records')

with open('results_jsonlines.json', 'w', encoding='utf8') as fout:
    fout.write("\n".join([
        json.dumps(row, ensure_ascii=False)
        for row in results
    ]))